## Imports

In [2]:
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import urllib3
import re
from textwrap import shorten
import os

In [3]:
df = pd.read_csv('South_Florida_resi_flips_since_start_of_2022 - Sheet1.csv')

## Image URL collector

In [3]:
# Drop last n rows of a df. In case there are sums that need clipped
# df.drop(df.tail(n).index, inplace=True)

In [5]:
df['image_links'] = ''
links = []
for url in df['story_link']:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        images = soup.find_all('img')
        counter = 0
        for image in images:
            counter += 1
            if 'jpg' in image['src']:
                links.append(image['src'])
                if counter == len(images):
                    links.append('no image found')
                break
    except:
        links.append('no_image')

df['image_links'] = links

## Article link formatter

In [6]:
len(links)

21

In [9]:
df["short_description"] = df["full_address"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['full_address'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["story_link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0     <a href="https://therealdeal.com/miami/2022/01...
1     <a href="https://therealdeal.com/miami/2022/04...
2     <a href="https://therealdeal.com/miami/2022/05...
3     <a href="https://therealdeal.com/miami/2022/02...
4     <a href="https://therealdeal.com/miami/2022/07...
5     <a href="https://therealdeal.com/miami/2022/02...
6     <a href="https://therealdeal.com/miami/2022/08...
7     <a href="https://therealdeal.com/miami/2022/03...
8     <a href="https://therealdeal.com/miami/2022/07...
9     <a href="https://therealdeal.com/miami/2022/05...
10    <a href="https://therealdeal.com/miami/2022/06...
11    <a href="https://therealdeal.com/miami/2022/04...
12    <a href="https://therealdeal.com/miami/2022/06...
13    <a href="https://therealdeal.com/miami/2022/06...
14    <a href="https://therealdeal.com/miami/2022/05...
15    <a href="https://therealdeal.com/miami/2022/07...
16    <a href="https://therealdeal.com/miami/2022/01...
17    <a href="https://therealdeal.com/miami/202

In [10]:
df.description_link[0]

'<a href="https://therealdeal.com/miami/2022/01/18/title-insurance-honcho-pays-15m-for-restored-waterfront-mid-miami-beach-house/" target="_blank" rel="noopener noreferrer">544 Lakeview Drive,</a> Miami Beach, FL'

## Google Maps API Geolocater Setup

In [11]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [12]:
df['geo_address'] = df['full_address']
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

## Correction Section

In [13]:
df.columns

Index(['full_address', 'buyer', 'seller', 'old_sale_price', 'old_sale_date',
       'new_sale_price', 'new_sale_date', 'percent_change', 'description',
       'story_link', 'image_links', 'short_description', 'remaining_desc',
       'description_link', 'geo_address', 'loc', 'point', 'lat', 'lon',
       'altitude'],
      dtype='object')

In [14]:
df

,full_address,buyer,seller,old_sale_price,old_sale_date,new_sale_price,new_sale_date,percent_change,description,story_link,image_links,short_description,remaining_desc,description_link,geo_address,loc,point,lat,lon,altitude
0,"544 Lakeview Drive, Miami Beach, FL",Jarett and Elizabeht Fein,Barry Brodsky,"$3,800,000.00",May 2020,"$15,300,000.00",January 2022,302.631579,NaN,https://therealdeal.com/miami/2022/01/18/title...,https://s14.therealdeal.com/trd/m/up/2022/01/M...,"544 Lakeview Drive,","Miami Beach, FL","<a href=""https://therealdeal.com/miami/2022/01...","544 Lakeview Drive, Miami Beach, FL","(544 Lakeview Dr, Miami Beach, FL 33140, USA, ...","(25.8238246, -80.12746159999999, 0.0)",25.823825,-80.127462,0.0
1,"6455 Pine Tree Drive Circle, Miami Beach, FL",unknown,Steve Witkoff,"$7,000,000.00",January 2021,"$20,000,000.00",April 2022,185.714286,NaN,https://therealdeal.com/miami/2022/04/05/steve...,https://s13.therealdeal.com/trd/m/up/2022/04/m...,6455 Pine Tree Drive,"Circle, Miami Beach, FL","<a href=""https://therealdeal.com/miami/2022/04...","6455 Pine Tree Drive Circle, Miami Beach, FL","(6455 Pine Tree Dr Cir, Miami Beach, FL 33141,...","(25.8464182, -80.1261529, 0.0)",25.846418,-80.126153,0.0
2,"215 Indian Road, Palm Beach, FL",Sandra and Paul Edgerley,Todd Michael Glaser,"$6,400,000.00",April 2021,"$15,500,000.00",May 2022,142.187500,NaN,https://therealdeal.com/miami/2022/05/27/todd-...,https://s11.therealdeal.com/trd/m/up/2022/05/M...,"215 Indian Road,","Palm Beach, FL","<a href=""https://therealdeal.com/miami/2022/05...","215 Indian Road, Palm Beach, FL","(215 Indian Rd, Palm Beach, FL 33480, USA, (26...","(26.7699831, -80.0398262, 0.0)",26.769983,-80.039826,0.0
3,"6300 North Bay Road, Miami Beach, FL",unknown,Michelle Simkins and Jason Rubell,"$9,700,000.00",January 2021,"$21,000,000.00",January 2022,116.494845,NaN,https://therealdeal.com/miami/2022/02/08/miche...,https://s14.therealdeal.com/trd/m/up/2022/02/m...,"6300 North Bay Road,","Miami Beach, FL","<a href=""https://therealdeal.com/miami/2022/02...","6300 North Bay Road, Miami Beach, FL","(6300 N Bay Rd, Miami Beach, FL 33141, USA, (2...","(25.8437238, -80.12955529999999, 0.0)",25.843724,-80.129555,0.0
4,"12203 Tillinghast Circle, Palm Beach Gardens, FL",unknown,Steven Frankel,"$11,000,000.00",June 2021,"$22,500,000.00",June 2022,104.545455,NaN,https://therealdeal.com/miami/2022/07/15/palm-...,https://s13.therealdeal.com/trd/m/up/2022/07/m...,12203 Tillinghast,"Circle, Palm Beach Gardens, FL","<a href=""https://therealdeal.com/miami/2022/07...","12203 Tillinghast Circle, Palm Beach Gardens, FL","(12203 Tillinghast Cir, Palm Beach Gardens, FL...","(26.8559188, -80.1211601, 0.0)",26.855919,-80.121160,0.0
5,"622 North Flagler Drive, West Palm Beach, FL",William Segel,Pat Carney,"$5,000,000.00",May 2021,"$10,200,000.00",January 2022,104.000000,NaN,https://therealdeal.com/miami/2022/02/04/real-...,https://s12.therealdeal.com/trd/m/up/2022/02/m...,622 North Flagler,"Drive, West Palm Beach, FL","<a href=""https://therealdeal.com/miami/2022/02...","622 North Flagler Drive, West Palm Beach, FL","(622 N Flagler Dr, West Palm Beach, FL 33401, ...","(26.7192802, -80.0496625, 0.0)",26.719280,-80.049662,0.0
6,"14958 Palmwood Road, Palm Beach Gardens, FL",Robert and Myron Miller,Aaron Levine,"$5,882,750.00",July 2021,"$11,000,000.00",July 2022,86.987378,NaN,https://therealdeal.com/miami/2022/08/02/devel...,https://s14.therealdeal.com/trd/m/up/2022/08/m...,"14958 Palmwood Road,","Palm Beach Gardens, FL","<a href=""https://therealdeal.com/miami/2022/08...","14958 Palmwood Road, Palm Beach Gardens, FL","(14958 Palmwood Rd, Palm Beach Gardens, FL 334...","(26.8964016, -80.0759735, 0.0)",26.896402,-80.075974,0.0
7,"2929 Ocean Trust, Gulf Stream, FL",David J. Miller,Jesse Aron Cohn and Karen Sue Cohn,"$15,900,000.00",May 2021,"$26,700,000.00",March 2022,67.924528,NaN,https://therealdeal.com/miami/2022/03/24/ellio...,https://s11.therealdeal.com/trd/m/up/2022/03/m...,"2929 Ocean 

## Correction section

In [13]:
# df.at[0,'Net decline in footprint, in sf']=('Almost 200K sf')
# df.at[1,'Address(es) of space company vacated or plans to vacate, in sf']=('690 East Middlefield Road, Mountain View. Synopsys is subletting the entire building to Waymo.')

## HTML popup formatter

In [15]:
df.columns

Index(['full_address', 'buyer', 'seller', 'old_sale_price', 'old_sale_date',
       'new_sale_price', 'new_sale_date', 'percent_change', 'description',
       'story_link', 'image_links', 'short_description', 'remaining_desc',
       'description_link', 'geo_address', 'loc', 'point', 'lat', 'lon',
       'altitude'],
      dtype='object')

In [25]:
def popup_html(row):
    i = row
    address = df['full_address'].iloc[i]
    percent_change = df['percent_change'].iloc[i]
    buyer = df['buyer'].iloc[i]
    seller = df['seller'].iloc[i]
    old_sale_price = df['old_sale_price'].iloc[i]
    old_sale_date = df['old_sale_date'].iloc[i]
    new_sale_price = df['new_sale_price'].iloc[i]
    new_sale_date = df['new_sale_date'].iloc[i]
    image = df['image_links'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <img src={} width="256" height="156">'''.format(image) + '''<br>______________________________________<br>
    Buyer: <strong>{}'''.format(buyer) + '''</strong><br>
    Seller: <strong>{}'''.format(seller) + '''</strong><br>
    Address: <em>{}'''.format(address) + '''</em><br>
    Percent Change: <strong>{}'''.format(percent_change) + '''<strong><br>
    Previous Sale Price: <strong>{}'''.format(old_sale_price) + '''<strong><br>
    New Sale Price: <strong>{}'''.format(new_sale_price) + '''<strong><br>
    Previous Sale Date: <strong>{}'''.format(old_sale_date) + '''<strong><br>
    New Sale Date: <strong>{}'''.format(new_sale_date) + '''<strong><br>
    </html>
    '''
    return html


### HTML reservoir

In [16]:
#     Developer(s): <strong><em>{}'''.format(developer) + '''</strong></em><br><br>

## Map Maker

In [17]:
df.description_link[0]

'<a href="https://therealdeal.com/miami/2022/01/18/title-insurance-honcho-pays-15m-for-restored-waterfront-mid-miami-beach-house/" target="_blank" rel="noopener noreferrer">544 Lakeview Drive,</a> Miami Beach, FL'

In [26]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=8)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Top South Florida Resi Sale Flips Since January')

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [19]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [20]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [21]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Chicago_resi_map_01.01.22_7.13.22
